In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as ex
import bz2
import func as F
import torch
import scipy.stats as stats
import scipy
from plotly.subplots import make_subplots
import plotly.io as pio

In [2]:
ae_res = F.load("G:/MEX/results/ae_dfs.pbz2")
ref_res = F.load("G:/MEX/results/ref_dfs.pbz2")
transfer_res = F.load("G:/MEX/results/transfer_dfs.pbz2")

reduc_ref_res = F.load("G:/MEX/results/ref_reduc_dfs.pbz2")
reduc_res = F.load("G:/MEX/results/reduc_dfs.pbz2")

In [3]:
def append(source, target, s_keys, t_keys):
    for sk, tk  in zip(s_keys, t_keys):
        if sk not in source:
            target[tk].append(0)
        else:
            target[tk].append(np.mean([val for val in source[sk]]))
def mean_confidence_interval(data, confidence=0.95):
    data = np.nan_to_num(data, 0)
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

# Model specs

In [23]:
dfs = {}
dfs["Name"] = []
dfs["Parameters"] = []
dfs["Memory"] = []
dfs["Inference time"] = []

# AE
s_keys = ["name", "params", "mem", "time"]
t_keys = ["Name", "Parameters", "Memory", "Inference time"]

ae = ae_res[0]
append(ae, dfs, s_keys, t_keys)

s_keys = ["params", "mem", "time"]
t_keys = ["Parameters", "Memory", "Inference time"]
ref_res1 = ref_res[0]
for k, v in ref_res1.items():
    append(v, dfs, s_keys, t_keys)
    dfs["Name"].append(k)

ref_res1 = reduc_ref_res[0]
for k, v in ref_res1.items():
    append(v, dfs, s_keys, t_keys)
    dfs["Name"].append(k+"_reduc")

dfss = pd.DataFrame(dfs)
dfss.to_csv("../results/model_spec.csv")

# Peformance data


In [66]:
perf_dfs = {}
t_keys = ["Reconstruction error", "Adversarial error", "Rotational error",
          "Sum of rotation angle x", "Sum of rotation angle y",
          "Avg Sum position cost x", "Avg Sum position cost y",
          "Avg Sum rotation cost x", "Avg Sum rotation cost y",]
s_keys = ["recon_err", "adv_err", "rot_err", "sum_delta_rot_x", "sum_delta_rot_y",
          "avg_sum_pCost_x","avg_sum_pCost_y","avg_sum_rCost_x","avg_sum_rCost_t"]

perf_dfs["Name"] = []
for k in t_keys:
    perf_dfs[k] = []

for ae in ae_res:
    perf_dfs["Name"].append(ae["name"][0])
    append(ae, perf_dfs, s_keys, t_keys)

for i,domain in enumerate(ref_res):
    for k,v in domain.items():
        append(v, perf_dfs, s_keys, t_keys)
        perf_dfs["Name"].append(k+"_R"+str(i+1))

for i,domain in enumerate(transfer_res):
    for k,v in domain.items():
        append(v, perf_dfs, s_keys, t_keys)
        perf_dfs["Name"].append("Transfer_"+k+"_R"+str(i+1))

for i,domain in enumerate(reduc_ref_res):
    for k,v in domain.items():
        append(v, perf_dfs, s_keys, t_keys)
        perf_dfs["Name"].append("Reduc_"+k+"_R"+str(i+1))

for i,domain in enumerate(reduc_res):
    for k,v in domain.items():
        append(v, perf_dfs, s_keys, t_keys)
        perf_dfs["Name"].append("Reduc_Transfer_"+k+"_R"+str(i+1))


In [236]:
perf_dfss = pd.DataFrame(perf_dfs)
perf_dfss.to_csv("../results/perf_data.csv")


NameError: name 'perf_dfs' is not defined

# Generation performance

## Reference models

In [4]:
def prepare(dfs, source_keys, target_keys, models):
    results = {}
    results["x"] = models
    n = float(len(dfs))
    for k in target_keys: results[k] = np.zeros((len(models), 3))
    for model_id,m in enumerate(models):
        for domain in dfs:
            for key_id,sk in enumerate(source_keys):
                vals = domain[m][sk]
                try:
                    if np.isnan(vals).sum() > 0:
                        vals = np.nan_to_num(vals, 0)
                except:
                    vals = vals[0]
                err, low, high = mean_confidence_interval(vals)
                results[target_keys[key_id]][model_id] += np.asarray([err, low, high]) / n
    return results

def prepare_per_rig(dfs, source_keys, target_keys, models):
    tot_results = []
    for domain in dfs:
        results = {}
        results["x"] = models
        for k in target_keys: results[k] = np.zeros((len(models), 3))
        for model_id,m in enumerate(models):
            for key_id,sk in enumerate(source_keys):
                vals = domain[m][sk]
                try:
                    if np.isnan(vals).sum() > 0:
                        vals = np.nan_to_num(vals, 0)
                except:
                    vals = vals[0]
                err, low, high = mean_confidence_interval(vals)
                results[target_keys[key_id]][model_id] += np.asarray([err, low, high])
        tot_results.append(results)
    return tot_results

def check_vals(vals):
    try:
        if np.isnan(vals).sum() > 0:
            vals = np.nan_to_num(vals, 0)
    except:
        vals = vals[0]
    return vals

def perpare_transfer_data_per_rig(models, s_keys, t_keys):
    f_perf_over_raw = []
    f_perf_over_ref = []

    t_perf_over_raw = []
    t_perf_over_ref = []

    np.random.seed(2021)

    for d_id in range(n):
        results = [{}, {}, {}, {}]
        
        for res in results:
            for m in models: res[m] = {"p_val":np.zeros(3), "whitney_p_val":np.zeros(3)}

        for model_id,m in enumerate(models):
            for key_id,sk in enumerate(s_keys):
                ref_vals = check_vals(ref_res[d_id][m][sk].values)
                r_vals = check_vals(transfer_res[d_id][m+"_RAW"][sk].values)
                f_vals = check_vals(transfer_res[d_id][m+"_F"][sk].values)
                t_vals = check_vals(transfer_res[d_id][m+"_T"][sk].values)

                if key_id == 0:
                    results[0][m]["p_val"][:] = stats.ttest_ind(f_vals, r_vals, equal_var=False).pvalue
                    results[1][m]["p_val"][:] = stats.ttest_ind(f_vals, ref_vals, equal_var=False).pvalue
                    results[2][m]["p_val"][:] = stats.ttest_ind(t_vals, r_vals, equal_var=False).pvalue
                    results[3][m]["p_val"][:] = stats.ttest_ind(t_vals, ref_vals, equal_var=False).pvalue
      
                    results[0][m]["whitney_p_val"][:] = stats.mannwhitneyu(f_vals, r_vals).pvalue
                    results[1][m]["whitney_p_val"][:] = stats.mannwhitneyu(f_vals, ref_vals).pvalue
                    results[2][m]["whitney_p_val"][:] = stats.mannwhitneyu(t_vals, r_vals).pvalue
                    results[3][m]["whitney_p_val"][:] = stats.mannwhitneyu(t_vals, ref_vals).pvalue
    

                fr_vals = np.nan_to_num((r_vals - f_vals)/(r_vals), 0) * 100
                fref_vals = np.nan_to_num((ref_vals - np.random.choice(f_vals, len(ref_vals)))/(ref_vals), 0) * 100
                tr_vals = np.nan_to_num((r_vals - t_vals)/(r_vals), 0) * 100
                tref_vals = np.nan_to_num((ref_vals - np.random.choice(t_vals, len(ref_vals)))/(ref_vals), 0) * 100
                
                
                results[0][m][sk] = fr_vals
                results[1][m][sk] = fref_vals
                results[2][m][sk] = tr_vals
                results[3][m][sk] = tref_vals

        f_perf_over_raw.append(results[0])
        f_perf_over_ref.append(results[1])
        t_perf_over_raw.append(results[2])
        t_perf_over_ref.append(results[3])
    return f_perf_over_raw, f_perf_over_ref, t_perf_over_raw, t_perf_over_ref, results


In [5]:
s_keys = ["recon_err", "adv_err", "rot_err", 
            "sum_delta_rot_x", "sum_delta_rot_y", 
            "avg_sum_pCost_x", "avg_sum_pCost_y",
            "avg_sum_rCost_x", "avg_sum_rCost_t"]

t_keys = ["Reconstruction error", "Adversarial error", "Rotational error",
            "Sum angular update x", "Sum angular update y",
            "AOC position cost x", "AOC position cost y",
            "AOC rotation cost x", "AOC rotation cost y"]

t_keys2 = ["Reconstruction error", "Adversarial error", "Rotational error",
            "Sum angular update x"]

# models = ["AE+MoE", "AE+LSTM"]
# results = prepare(ref_res, s_keys, t_keys, models)


In [6]:
def plot_bar(x, results, title="", x_axis_name=None, y_axis_name=None, width=500, height=250):
    fig = go.Figure()
    swatches = ex.colors.qualitative.Safe
    
    for j, model in enumerate(results["x"]):
        fig.add_trace(go.Bar(name=model, x=x, y=[results[i][j][0] for i in x],
                                marker_color=swatches[j],
                                error_y=dict(type="data", array=[results[i][j][2] for i in x], arrayminus=[results[i][j][1] for i in x])))
    
    fig.update_layout(
        title_text=title,
        width = width, height=height, 
        font_family="Serif", font_size=14, 
        margin_l=5, margin_t=40, margin_b=5, margin_r=5)
    if x_axis_name is not None:
        fig.update_xaxes(
            title_text=x_axis_name
        )
    if y_axis_name is not None:
        fig.update_yaxes(
            title_text=y_axis_name
        )
    return fig

def plot_bar_twin(x, results, title="", x_axis_name=None, y_axis_name=None, width=500, height=250, skip_first=False):
    fig = go.Figure()
    swatches = ex.colors.qualitative.Safe
    
    category = x[0]
    
    if skip_first:
        tup = zip(results["x"][1:-1:2],results["x"][2::2])
        m1 = results["x"][0]
        model = m1.split("-")[0]
        fig.add_trace(go.Bar(name=model, x=[m1], y=[results[category][0][0]],
                                marker_color=swatches[-1],
                                error_y=dict(type="data", array=[results[category][0][2]], arrayminus=[results[category][0][1]])))
        k = 1
    else:
        tup = zip(results["x"][:-1:2],results["x"][1::2])
        k = 0
    
    for j, (m1, m2) in enumerate(tup):
        model = m1.split("-")[0]
        i = j*2 + k
        fig.add_trace(go.Bar(name=model, x=[m1,m2], y=[results[category][i][0],results[category][i+1][0]],
                                marker_color=swatches[i],
                                error_y=dict(type="data", array=[results[category][i][2],results[category][i+1][2]], arrayminus=[results[category][i][1],results[category][i+1][1]])))
    
    fig.update_layout(
        title_text=title,
        width = width, height=height, 
        showlegend=False,
        font_family="Serif", font_size=14, 
        margin_l=5, margin_t=40, margin_b=5, margin_r=5)
    if x_axis_name is not None:
        fig.update_xaxes(
            title_text=x_axis_name
        )
    if y_axis_name is not None:
        fig.update_yaxes(
            title_text=y_axis_name
        )
    return fig

def plot_bar_multi_stack(x, results1, results2, title="", x_axis_name=None, y_axis_name=None,  rows=1, cols=1, width=500, height=250):
    fig = make_subplots(rows=rows, cols=cols)
    swatches = ex.colors.qualitative.Safe
    r, c = 1,1
    if rows * cols != len(x):
        raise ValueError("Wrong Rows or Cols")
    categories = x
    results = results1
    for k, key in enumerate(categories):
        x = [key]
        for j, model in enumerate(results["x"]):
            if k > 0:
                fig.add_trace(go.Bar(x=x, y=[results[i][j][0] for i in x], marker_color=swatches[j], error_y=dict(type="data", array=[results[i][j][2] for i in x], arrayminus=[results[i][j][1] for i in x]), showlegend=False), row=r, col=c, )
            else:
                fig.add_trace(go.Bar(name=model, x=x, y=[results[i][j][0] for i in x], marker_color=swatches[j], error_y=dict(type="data", array=[results[i][j][2] for i in x], arrayminus=[results[i][j][1] for i in x])), row=r, col=c)
        c += 1
        if c > cols:
            c = 1
            r += 1
    l = j + 1
    results = results2
    for k, key in enumerate(categories):
        x = [key]
        for j, model in enumerate(results["x"]):
            if k > 0:
                fig.add_trace(go.Bar(x=x, y=[results[i][j][0] for i in x], marker_color=swatches[j+l], error_y=dict(type="data", array=[results[i][j+l][2] for i in x], arrayminus=[results[i][j+l][1] for i in x]), showlegend=False), row=r, col=c, )
            else:
                fig.add_trace(go.Bar(name=model, x=x, y=[results[i][j][0] for i in x], marker_color=swatches[j+l], error_y=dict(type="data", array=[results[i][j][2] for i in x], arrayminus=[results[i][j+l][1] for i in x])), row=r, col=c)
        c += 1
        if c > cols:
            c = 1
            r += 1

    fig.update_layout(
        title_text=title,
        barmode="stack",
        width = width, height=height, 
        font_family="Serif", font_size=14, 
        margin_l=5, margin_t=40, margin_b=5, margin_r=5)
    if x_axis_name is not None:
        for i, names in enumerate(x_axis_name):
            for j, axis_name in enumerate(names):
                if axis_name is not None:
                    fig.update_xaxes(
                        title_text=axis_name, row=i+1, col=j+1,
                        title_standoff = 3
                    )
    if y_axis_name is not None:
        for i, names in enumerate(y_axis_name):
            for j, axis_name in enumerate(names):
                if axis_name is not None:
                    fig.update_yaxes(
                        title_text=axis_name, row=i+1, col=j+1,
                        title_standoff = 3
                    )
    return fig
def plot_bar_multi(x, results, title="", x_axis_name=None, y_axis_name=None,  rows=1, cols=1, width=500, height=250):
    fig = make_subplots(rows=rows, cols=cols)
    swatches = ex.colors.qualitative.Safe
    r, c = 1,1
    if rows * cols != len(x):
        raise ValueError("Wrong Rows or Cols")
    categories = x
    for k, key in enumerate(categories):
        x = [key]
        for j, model in enumerate(results["x"]):
            if k > 0:
                fig.add_trace(go.Bar(x=x, y=[results[i][j][0] for i in x], marker_color=swatches[j], error_y=dict(type="data", array=[results[i][j][2] for i in x], arrayminus=[results[i][j][1] for i in x]), showlegend=False), row=r, col=c, )
            else:
                fig.add_trace(go.Bar(name=model, x=x, y=[results[i][j][0] for i in x], marker_color=swatches[j], error_y=dict(type="data", array=[results[i][j][2] for i in x], arrayminus=[results[i][j][1] for i in x])), row=r, col=c)
        c += 1
        if c > cols:
            c = 1
            r += 1

    fig.update_layout(
        title_text=title,
        width = width, height=height, 
        font_family="Serif", font_size=14, 
        margin_l=5, margin_t=40, margin_b=5, margin_r=5)
    if x_axis_name is not None:
        for i, names in enumerate(x_axis_name):
            for j, axis_name in enumerate(names):
                if axis_name is not None:
                    fig.update_xaxes(
                        title_text=axis_name, row=i+1, col=j+1,
                        title_standoff = 1
                    )
    if y_axis_name is not None:
        for i, names in enumerate(y_axis_name):
            for j, axis_name in enumerate(names):
                if axis_name is not None:
                    fig.update_yaxes(
                        title_text=axis_name, row=i+1, col=j+1,
                        title_standoff = 1
                    )
    return fig
def plot_radar(x, results, title="", x_axis_name=None, y_axis_name=None, width=500, height=250):
    fig = go.Figure()
    
    swatches = ex.colors.qualitative.Safe
    categories = x
    for j, model in enumerate(results["x"]):
        fig.add_trace(go.Scatterpolar(
            r=[results[cat][j][0] for cat in categories],
            theta=categories,
            marker_color=swatches[j],
            name=model
        ))
    fig.update_layout(
        title_text=title,
        width = width, height=height, 
        font_family="Serif", font_size=14, 
        margin_l=5, margin_t=40, margin_b=5, margin_r=5)
    if x_axis_name is not None:
        fig.update_xaxes(
            title_text=x_axis_name
        )
    if y_axis_name is not None:
        fig.update_yaxes(
            title_text=y_axis_name
        )
    return fig


In [323]:
fig = plot_bar_multi(x=t_keys2, results=results, title="OMG generation performance", y_axis_name=[["MSE", "ADV loss", "MSE"],["Angular update in radian/s", "m", "radian"]], width=800, height=400, rows=2, cols=2)

In [324]:
fig.show()

In [18]:
# models = ["AE+MoE", "AE+LSTM"]
# model_list = list(ref_res[0].keys()) 
# models2 = [name for name in model_list if "+LSTM" not in name]
models2 = ["AE+MoE", "RBF-CAT+MoE", "VAE-CAT+MoE"]
models2_t = [m+"_T" for m in models2]

ref_results = prepare(ref_res, s_keys, t_keys, models2)
transfer_results = prepare(transfer_res, s_keys, t_keys, models2_t)
reduc_results = prepare(reduc_res, s_keys, t_keys, models2_t)

In [24]:
title = "Reconstruction error comparison between the three RAE with the three best performing models"
width = 400
height = 400
fig = go.Figure()
swatches = ex.colors.qualitative.Safe

def get_y(x, i, j):
    return [
        ref_results[x][i][j],
        transfer_results[x][i][j],
        reduc_results[x][i][j],
    ]

x = t_keys2[0]
n = 3
labels = ["ref", "FE/FC", "FS"]
show_legend = True
for j in range(3):
    if j > 0: show_legend = False
    fig.add_trace(go.Bar(name=models[j], x=labels, y=get_y(x,j,0),
                            marker_color=swatches[j],
                            error_y=dict(type="data", array=get_y(x,j,2), arrayminus=get_y(x,j,1))))

fig.update_layout(
    title_text=title,
    width = width, height=height, 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)

fig.update_yaxes(
    title_text="MSE"
)

In [16]:
fig = plot_bar([t_keys2[0]], ref_results, width=500, height=300,
                title="Reference OMG: Reconstruction error", y_axis_name="MSE")
# def plot_bar(x, results, title="", x_axis_name=None, y_axis_name=None, width=500, height=250):

In [17]:
fig.show()

In [50]:
fig.show()

In [51]:
fig = plot_bar_twin([t_keys2[1]], results, width=500, height=300,
                title="Reference OMG: Adversarial error", y_axis_name="Adv. loss", skip_first=True)
fig.show()

In [52]:
fig = plot_bar_twin([t_keys2[2]], results, width=500, height=300,
                title="Reference OMG: Rotational error", y_axis_name="MSE", skip_first=True)
fig.show()

In [53]:
fig = plot_bar_twin([t_keys2[3]], results, width=500, height=300,
                title="Reference OMG: Sum of angular updates", y_axis_name="radian/s", skip_first=True)
fig.show()

In [65]:
fig = plot_bar_twin([t_keys2[4]], results, width=500, height=300,
                title="Reference FC-MoGen: AOC position cost", y_axis_name="m")
fig.show()

In [66]:
fig = plot_bar_twin([t_keys2[5]], results, width=500, height=300,
                title="Reference FC-MoGen: AOC rotation cost", y_axis_name="radian")
fig.show()

In [45]:
results_rig = prepare_per_rig(ref_res, s_keys, t_keys, models2)

symbols = ["cross", "square", "circle", "diamond"]
marker_size = 10
fig = go.Figure()
swatches = ex.colors.qualitative.Safe
categories = ["R"+str(i) for i in range(1, 6)]
k = 0
for j, model in enumerate(results["x"]):
    if "-IN" in model or "LSTM" in model: continue
    fig.add_trace(go.Scatterpolar(
        r=[results_rig[cat][t_keys2[0]][j][0] for cat in range(5)],
        theta=categories,
        marker_symbol=symbols[k],
        marker_size=marker_size,
        marker_color=swatches[j],
        name=model
    ))
    k+= 1
fig.update_layout(
    title_text="Reference OMG: Reconstruction error",
    width = 400, height=400,
    legend=dict(orientation="h"), 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)
# if x_axis_name is not None:
#     fig.update_xaxes(
#         title_text=
#     )
fig.update_yaxes(
    title_text="MSE"
)

# Transferering quality


In [53]:
models = list(transfer_res[0].keys())
print(models)

['AE+LSTM_RAW', 'AE+MoE_RAW', 'DEC-CAT+MoE_RAW', 'DEC-IN+MoE_RAW', 'RBF-IN+LSTM_RAW', 'RBF-CAT+LSTM_RAW', 'RBF-CAT+MoE_RAW', 'RBF-IN+MoE_RAW', 'VAE-IN+LSTM_RAW', 'VAE-CAT+LSTM_RAW', 'VAE-CAT+MoE_RAW', 'VAE-IN+MoE_RAW', 'AE+LSTM_F', 'AE+LSTM_T', 'AE+MoE_F', 'AE+MoE_T', 'DEC-CAT+MoE_F', 'DEC-CAT+MoE_T', 'DEC-IN+MoE_F', 'DEC-IN+MoE_T', 'RBF-CAT+LSTM_T', 'RBF-CAT+LSTM_F', 'RBF-IN+LSTM_F', 'RBF-IN+LSTM_T', 'RBF-CAT+MoE_F', 'RBF-CAT+MoE_T', 'RBF-IN+MoE_F', 'RBF-IN+MoE_T', 'VAE-CAT+LSTM_T', 'VAE-CAT+LSTM_F', 'VAE-IN+LSTM_F', 'VAE-IN+LSTM_T', 'VAE-CAT+MoE_F', 'VAE-CAT+MoE_T', 'VAE-IN+MoE_F', 'VAE-IN+MoE_T']


In [79]:
raw_perfs = {}
frozen_perfs = {}
trained_perfs = {}
n = len(transfer_res)
for m in models:
    if "RAW" in m:
        raw_perfs[m] = [transfer_res[i][m] for i in range(n)]
    elif "_F" in m:
        frozen_perfs[m] = [transfer_res[i][m] for i in range(n)]
    elif "_T" in m:
        trained_perfs[m] = [transfer_res[i][m] for i in range(n)]

In [94]:
raw_models = list(raw_perfs.keys())
frozen_models = list(frozen_perfs.keys())
trained_models = list(trained_perfs.keys())
print(sorted(raw_models))
print(sorted(frozen_models))
print(sorted(trained_models))

['AE+LSTM_RAW', 'AE+MoE_RAW', 'DEC-CAT+MoE_RAW', 'DEC-IN+MoE_RAW', 'RBF-CAT+LSTM_RAW', 'RBF-CAT+MoE_RAW', 'RBF-IN+LSTM_RAW', 'RBF-IN+MoE_RAW', 'VAE-CAT+LSTM_RAW', 'VAE-CAT+MoE_RAW', 'VAE-IN+LSTM_RAW', 'VAE-IN+MoE_RAW']
['AE+LSTM_F', 'AE+MoE_F', 'DEC-CAT+MoE_F', 'DEC-IN+MoE_F', 'RBF-CAT+LSTM_F', 'RBF-CAT+MoE_F', 'RBF-IN+LSTM_F', 'RBF-IN+MoE_F', 'VAE-CAT+LSTM_F', 'VAE-CAT+MoE_F', 'VAE-IN+LSTM_F', 'VAE-IN+MoE_F']
['AE+LSTM_T', 'AE+MoE_T', 'DEC-CAT+MoE_T', 'DEC-IN+MoE_T', 'RBF-CAT+LSTM_T', 'RBF-CAT+MoE_T', 'RBF-IN+LSTM_T', 'RBF-IN+MoE_T', 'VAE-CAT+LSTM_T', 'VAE-CAT+MoE_T', 'VAE-IN+LSTM_T', 'VAE-IN+MoE_T']


In [96]:
list(ref_res[0].keys())


['AE+MoE',
 'AE+LSTM',
 'RBF-CAT+LSTM',
 'RBF-IN+LSTM',
 'RBF-CAT+MoE',
 'RBF-IN+MoE',
 'VAE-CAT+LSTM',
 'VAE-IN+LSTM',
 'VAE-CAT+MoE',
 'VAE-IN+MoE',
 'DEC-CAT+MoE',
 'DEC-IN+MoE']

In [99]:
a = np.random.normal(0,1, 100)
b = np.random.normal(0,2, 100)

rrrr = stats.mannwhitneyu(a,b)
rrrr.pvalue

0.10391280831854444

In [239]:
models = [m.replace("_RAW","") for m in raw_models]
s_keys2 = s_keys.copy() + ["p_val", "whitney_p_val"]
t_keys3 = t_keys.copy()+ ["P value", "MWU P value"]
fr, fref, tr, tref,res  = perpare_transfer_data_per_rig(models, s_keys, t_keys)

<ipython-input-238-cc379f0e3b81>:80: RuntimeWarning:

invalid value encountered in true_divide

<ipython-input-238-cc379f0e3b81>:82: RuntimeWarning:

invalid value encountered in true_divide



In [240]:
fr_results = prepare(fr, s_keys2, t_keys3, models)
fref_results = prepare(fref, s_keys2, t_keys3, models)
tr_results = prepare(tr, s_keys2, t_keys3, models)
tref_results = prepare(tref, s_keys2, t_keys3, models)

In [241]:
print(fr_results.keys())

dict_keys(['x', 'Reconstruction error', 'Adversarial error', 'Rotational error', 'Sum angular update x', 'Sum angular update y', 'AOC position cost x', 'AOC position cost y', 'AOC rotation cost x', 'AOC rotation cost y', 'P value', 'MWU P value'])


In [246]:
f_dfs = {"name":models,  "pval":[], "mwu_pval":[]}
t_dfs = {"name":models,  "pval":[], "mwu_pval":[]}

for m_id, m in enumerate(models):
    f_dfs["pval"].append(fr_results["P value"][m_id][0])
    f_dfs["mwu_pval"].append(fr_results["MWU P value"][m_id][0])
    
    t_dfs["pval"].append(tr_results["P value"][m_id][0])
    t_dfs["mwu_pval"].append(tr_results["MWU P value"][m_id][0])

t_dfs = pd.DataFrame(t_dfs)
f_dfs = pd.DataFrame(f_dfs)

        

In [247]:
with pd.ExcelWriter("../results/stats_signif_trained.xlsx") as writer:  
    t_dfs.to_excel(writer)
with pd.ExcelWriter("../results/stats_signif_frozen.xlsx") as writer:  
    f_dfs.to_excel(writer)
    

In [251]:
fr_results["x"]

['AE+LSTM',
 'AE+MoE',
 'DEC-CAT+MoE',
 'DEC-IN+MoE',
 'RBF-IN+LSTM',
 'RBF-CAT+LSTM',
 'RBF-CAT+MoE',
 'RBF-IN+MoE',
 'VAE-IN+LSTM',
 'VAE-CAT+LSTM',
 'VAE-CAT+MoE',
 'VAE-IN+MoE']

In [326]:
ae_only = ["AE+MoE", "AE+LSTM"]
results = {"x":ae_only}

for k, v in fr_results.items():
    results[k] = v[:2]


fig = plot_bar_multi(x=t_keys2, results=results, title="(Frozen) FE-OMG performance improvement over cold-performance (%)", width=800, height=400, rows=2, cols=2)

In [327]:
fig.show()

In [328]:
ae_only = ["AE+MoE", "AE+LSTM"]
results = {"x":ae_only}

for k, v in fref_results.items():
    results[k] = v[:2]


fig = plot_bar_multi(x=t_keys2, results=results, title="(Frozen) FE-OMG performance improvement over reference-performance (%)", width=800, height=400, rows=2, cols=2)
fig.show()


In [329]:
ae_only = ["AE+MoE", "AE+LSTM"]
results = {"x":ae_only}

for k, v in tref_results.items():
    results[k] = v[:2]


fig = plot_bar_multi(x=t_keys2, results=results, title="(Trained) FE-OMG performance improvement over reference-performance (%)", width=800, height=400, rows=2, cols=2)
fig.show()


In [330]:
ae_only = ["AE+MoE", "AE+LSTM"]
results = {"x":ae_only}

for k, v in tr_results.items():
    results[k] = v[:2]


fig = plot_bar_multi(x=t_keys2, results=results, title="(Trained) FE-OMG performance improvement over cold-performance (%)", width=800, height=400, rows=2, cols=2)
fig.show()


In [331]:
ae_only = ["AE+MoE", "AE+LSTM"]
results = {"x":ae_only}

for k, v in tr_results.items():
    results[k] = [v[1],fr_results[k][1]]

results["x"][0] = "Frozen-" + results["x"][0]
results["x"][1] = "Trained-" +results["x"][1]

fig = plot_bar_multi(x=t_keys2, results=results, title="(vs cold-perf.) FE-OMG performance improvement between Frozen and Trained (%)", width=800, height=400, rows=2, cols=2)
fig.show()


In [332]:
ae_only = ["AE+MoE", "AE+LSTM"]
results = {"x":ae_only}

for k, v in tref_results.items():
    results[k] = [v[1],fref_results[k][1]]

results["x"][0] = "Frozen-" + results["x"][0]
results["x"][1] = "Trained-" +results["x"][1]

fig = plot_bar_multi(x=t_keys2, results=results, title="(vs ref-perf.) FE-OMG performance improvement between Frozen and Trained (%)", width=800, height=400, rows=2, cols=2)
fig.show()


array([[31.59630662, 30.99816745, 32.19444579],
       [46.38604759, 45.92515959, 46.8469356 ],
       [-9.00913915, -9.90531827, -8.11296004],
       [ 1.13868867,  0.39203833,  1.88533902],
       [ 1.29650186,  1.00789075,  1.58511297],
       [34.6296036 , 34.01529001, 35.2439172 ],
       [39.58645288, 39.08021226, 40.09269349],
       [ 2.80495046,  2.26190101,  3.3479999 ],
       [-2.14694084, -2.70429016, -1.58959151],
       [28.4299761 , 27.85811814, 29.00183407],
       [38.09942276, 37.58490043, 38.61394508],
       [21.85740083, 21.14071801, 22.57408365]])

In [339]:

fig = go.Figure()
swatches = ex.colors.qualitative.Safe
categories = fr_results["x"]
metric = t_keys2[0]
# for j, model in enumerate(results["x"]):
    # if "-IN" in model or "LSTM" in model: continue
marker_size = 15

fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in fr_results[metric]],
    theta=categories,
    marker_color=swatches[0],
    marker_symbol="circle",
    marker_size=marker_size,
    name="(Frozen) improvements over cold-perf."
))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in fref_results[metric]],
    theta=categories,
    marker_symbol="square",
    marker_size=marker_size,
    marker_color=swatches[1],
    name="(Frozen) improvements over ref-perf."
))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in tr_results[metric]],
    theta=categories,
    marker_symbol="diamond",
    marker_size=marker_size,
    marker_color=swatches[2],
    name="(Trained) improvements over cold-perf."
))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in tref_results[metric]],
    theta=categories,
    marker_symbol="cross",
    marker_size=marker_size,
    marker_color=swatches[3],
    name="(Trained) improvements over ref-perf."
))
fig.update_layout(
    title_text="Performance improvement over cold-/reference-performance (%)",
    width = 1000, height=800,
    legend=dict(orientation="h"), 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)
# if x_axis_name is not None:
#     fig.update_xaxes(
#         title_text=
#     )
fig.update_yaxes(
    title_text="MSE"
)

In [378]:
fig = go.Figure()
swatches = ex.colors.qualitative.Safe

categories = []
for m in fr_results["x"]:
    if ("LSTM" in m) or ("AE+" in m): continue
    categories.append(m)
metric = t_keys2[0]
# for j, model in enumerate(results["x"]):
    # if "-IN" in model or "LSTM" in model: continue
marker_size = 15

fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in fr_results[metric]],
    theta=categories,
    marker_color=swatches[1],
    marker_symbol="circle",
    marker_size=marker_size,
    name="(Frozen) improvements over cold-perf."
))
# fig.add_trace(go.Scatterpolar(
#     r=[res[0] for res in fref_results[metric]],
#     theta=categories,
#     marker_symbol="square",
#     marker_size=marker_size,
#     marker_color=swatches[1],
#     name="(Frozen) improvements over ref-perf."
# ))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in tr_results[metric]],
    theta=categories,
    marker_symbol="diamond",
    marker_size=marker_size,
    marker_color=swatches[3],
    name="(Trained) improvements over cold-perf."
))
# fig.add_trace(go.Scatterpolar(
#     r=[res[0] for res in tref_results[metric]],
#     theta=categories,
#     marker_symbol="cross",
#     marker_size=marker_size,
#     marker_color=swatches[3],
#     name="(Trained) improvements over ref-perf."
# ))
fig.update_layout(
    title_text="Reconstruction error improvement over cold-performance (%)",
    width = 1000, height=800,
    legend=dict(orientation="h"), 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)
# if x_axis_name is not None:
#     fig.update_xaxes(
#         title_text=
#     )
fig.update_yaxes(
    title_text="MSE"
)

In [375]:
fig = go.Figure()
swatches = ex.colors.qualitative.Safe

categories = []
for m in fr_results["x"]:
    if ("LSTM" in m) or ("AE+" in m): continue
    categories.append(m)
metric = t_keys2[0]
# for j, model in enumerate(results["x"]):
    # if "-IN" in model or "LSTM" in model: continue
marker_size = 15

# fig.add_trace(go.Scatterpolar(
#     r=[res[0] for res in fr_results[metric]],
#     theta=categories,
#     marker_color=swatches[0],
#     marker_symbol="circle",
#     marker_size=marker_size,
#     name="(Frozen) improvements over cold-perf."
# ))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in fref_results[metric]],
    theta=categories,
    marker_symbol="square",
    marker_size=marker_size,
    marker_color=swatches[1],
    name="(Frozen) improvements over ref-perf."
))
# fig.add_trace(go.Scatterpolar(
#     r=[res[0] for res in tr_results[metric]],
#     theta=categories,
#     marker_symbol="diamond",
#     marker_size=marker_size,
#     marker_color=swatches[2],
#     name="(Trained) improvements over cold-perf."
# ))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in tref_results[metric]],
    theta=categories,
    marker_symbol="cross",
    marker_size=marker_size,
    marker_color=swatches[3],
    name="(Trained) improvements over ref-perf."
))
fig.update_layout(
    title_text="Reconstruction error improvement over reference-performance (%)",
    width = 1000, height=800,
    legend=dict(orientation="h"), 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)
# if x_axis_name is not None:
#     fig.update_xaxes(
#         title_text=
#     )
fig.update_yaxes(
    title_text="MSE"
)

In [382]:
fig = go.Figure()
swatches = ex.colors.qualitative.Safe

categories = []
for m in fr_results["x"]:
    if ("LSTM" in m) or ("AE+" in m): continue
    categories.append(m)
metric = t_keys2[2]
# for j, model in enumerate(results["x"]):
    # if "-IN" in model or "LSTM" in model: continue
marker_size = 15

# fig.add_trace(go.Scatterpolar(
#     r=[res[0] for res in fr_results[metric]],
#     theta=categories,
#     marker_color=swatches[0],
#     marker_symbol="circle",
#     marker_size=marker_size,
#     name="(Frozen) improvements over cold-perf."
# ))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in fref_results[metric]],
    theta=categories,
    marker_symbol="square",
    marker_size=marker_size,
    marker_color=swatches[1],
    name="(Frozen) improvements over ref-perf."
))
# fig.add_trace(go.Scatterpolar(
#     r=[res[0] for res in tr_results[metric]],
#     theta=categories,
#     marker_symbol="diamond",
#     marker_size=marker_size,
#     marker_color=swatches[2],
#     name="(Trained) improvements over cold-perf."
# ))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in tref_results[metric]],
    theta=categories,
    marker_symbol="cross",
    marker_size=marker_size,
    marker_color=swatches[3],
    name="(Trained) improvements over ref-perf."
))
fig.update_layout(
    title_text="Rotational error improvement over reference-performance (%)",
    width = 1000, height=800,
    legend=dict(orientation="h"), 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)
# if x_axis_name is not None:
#     fig.update_xaxes(
#         title_text=
#     )
fig.update_yaxes(
    title_text="MSE"
)

In [383]:
fig = go.Figure()
swatches = ex.colors.qualitative.Safe

categories = []
for m in fr_results["x"]:
    if ("LSTM" in m) or ("AE+" in m): continue
    categories.append(m)
metric = t_keys2[2]
# for j, model in enumerate(results["x"]):
    # if "-IN" in model or "LSTM" in model: continue
marker_size = 15

fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in fr_results[metric]],
    theta=categories,
    marker_color=swatches[1],
    marker_symbol="circle",
    marker_size=marker_size,
    name="(Frozen) improvements over cold-perf."
))
# fig.add_trace(go.Scatterpolar(
#     r=[res[0] for res in fref_results[metric]],
#     theta=categories,
#     marker_symbol="square",
#     marker_size=marker_size,
#     marker_color=swatches[1],
#     name="(Frozen) improvements over ref-perf."
# ))
fig.add_trace(go.Scatterpolar(
    r=[res[0] for res in tr_results[metric]],
    theta=categories,
    marker_symbol="diamond",
    marker_size=marker_size,
    marker_color=swatches[3],
    name="(Trained) improvements over cold-perf."
))
# fig.add_trace(go.Scatterpolar(
#     r=[res[0] for res in tref_results[metric]],
#     theta=categories,
#     marker_symbol="cross",
#     marker_size=marker_size,
#     marker_color=swatches[3],
#     name="(Trained) improvements over ref-perf."
# ))
fig.update_layout(
    title_text="Rotational error improvement over cold-performance (%)",
    width = 1000, height=800,
    legend=dict(orientation="h"), 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)
# if x_axis_name is not None:
#     fig.update_xaxes(
#         title_text=
#     )
fig.update_yaxes(
    title_text="MSE"
)

In [364]:
results = {}
idx = [m_id for m_id, m in enumerate(fr_results["x"]) if ("AE+" not in m) and ("LSTM" not in m)]
for k, v in fr_results.items():
    results[k] = [v[i] for i in idx]
fig = plot_bar_twin([t_keys2[0]], results, width=500, height=300,
                title="F-FC-MoGen reconstruction error improvement", y_axis_name="%")
fig.show()

In [362]:
results = {}
idx = [m_id for m_id, m in enumerate(fr_results["x"]) if ("AE+" not in m) and ("LSTM" not in m)]
for k, v in fr_results.items():
    results[k] = [v[i] for i in idx]
fig = plot_bar_twin([t_keys2[1]], results, width=500, height=300,
                title="(Frozen) FC-MoGen improvement over cold-perf. ", y_axis_name="%")
fig.show()

In [363]:
results = {}
idx = [m_id for m_id, m in enumerate(fr_results["x"]) if ("AE+" not in m) and ("LSTM" not in m)]
for k, v in fr_results.items():
    results[k] = [v[i] for i in idx]
fig = plot_bar_twin([t_keys2[2]], results, width=500, height=300,
                title="(Frozen) FC-MoGen improvement over cold-perf. ", y_axis_name="%")
fig.show()

In [366]:
results = {}
idx = [m_id for m_id, m in enumerate(fr_results["x"]) if ("AE+" not in m) and ("LSTM" not in m)]
for k, v in fref_results.items():
    results[k] = [v[i] for i in idx]
fig = plot_bar_twin([t_keys2[0]], results, width=500, height=300,
                title="(Frozen) FC-MoGen improvement over ref-perf.", y_axis_name="%")
fig.show()

In [367]:
results = {}
idx = [m_id for m_id, m in enumerate(fr_results["x"]) if ("AE+" not in m) and ("LSTM" not in m)]
for k, v in fref_results.items():
    results[k] = [v[i] for i in idx]
fig = plot_bar_twin([t_keys2[1]], results, width=500, height=300,
                title="(Frozen) FC-MoGen improvement over ref-perf.", y_axis_name="%")
fig.show()

In [368]:
results = {}
idx = [m_id for m_id, m in enumerate(fr_results["x"]) if ("AE+" not in m) and ("LSTM" not in m)]
for k, v in fref_results.items():
    results[k] = [v[i] for i in idx]
fig = plot_bar_twin([t_keys2[2]], results, width=500, height=300,
                title="(Frozen) FC-MoGen improvement over ref-perf.", y_axis_name="%")
fig.show()

In [369]:
results = {}
idx = [m_id for m_id, m in enumerate(fr_results["x"]) if ("AE+" not in m) and ("LSTM" not in m)]
for k, v in fref_results.items():
    results[k] = [v[i] for i in idx]
fig = plot_bar_twin([t_keys2[3]], results, width=500, height=300,
                title="(Frozen) FC-MoGen improvement over ref-perf.", y_axis_name="%")
fig.show()

In [388]:
print(s_keys)
print(t_keys)

['recon_err', 'adv_err', 'rot_err', 'sum_delta_rot_x', 'sum_delta_rot_y', 'avg_sum_pCost_x', 'avg_sum_pCost_y', 'avg_sum_rCost_x', 'avg_sum_rCost_t']
['Reconstruction error', 'Adversarial error', 'Rotational error', 'Sum angular update x', 'Sum angular update y', 'AOC position cost x', 'AOC position cost y', 'AOC rotation cost x', 'AOC rotation cost y']


In [394]:
models = fref_results["x"]
print(models)

['AE+LSTM', 'AE+MoE', 'DEC-CAT+MoE', 'DEC-IN+MoE', 'RBF-IN+LSTM', 'RBF-CAT+LSTM', 'RBF-CAT+MoE', 'RBF-IN+MoE', 'VAE-IN+LSTM', 'VAE-CAT+LSTM', 'VAE-CAT+MoE', 'VAE-IN+MoE']


In [398]:
transfer_res[0].keys()

dict_keys(['AE+LSTM_RAW', 'AE+MoE_RAW', 'DEC-CAT+MoE_RAW', 'DEC-IN+MoE_RAW', 'RBF-IN+LSTM_RAW', 'RBF-CAT+LSTM_RAW', 'RBF-CAT+MoE_RAW', 'RBF-IN+MoE_RAW', 'VAE-IN+LSTM_RAW', 'VAE-CAT+LSTM_RAW', 'VAE-CAT+MoE_RAW', 'VAE-IN+MoE_RAW', 'AE+LSTM_F', 'AE+LSTM_T', 'AE+MoE_F', 'AE+MoE_T', 'DEC-CAT+MoE_F', 'DEC-CAT+MoE_T', 'DEC-IN+MoE_F', 'DEC-IN+MoE_T', 'RBF-CAT+LSTM_T', 'RBF-CAT+LSTM_F', 'RBF-IN+LSTM_F', 'RBF-IN+LSTM_T', 'RBF-CAT+MoE_F', 'RBF-CAT+MoE_T', 'RBF-IN+MoE_F', 'RBF-IN+MoE_T', 'VAE-CAT+LSTM_T', 'VAE-CAT+LSTM_F', 'VAE-IN+LSTM_F', 'VAE-IN+LSTM_T', 'VAE-CAT+MoE_F', 'VAE-CAT+MoE_T', 'VAE-IN+MoE_F', 'VAE-IN+MoE_T'])

In [418]:
ae_moe = prepare_per_rig(transfer_res, s_keys, t_keys, ["AE+MoE_F"])
rbf_moe = prepare_per_rig(transfer_res, s_keys, t_keys, ["RBF-CAT+MoE_F"])
reduc_moe = prepare_per_rig(reduc_res, s_keys, t_keys, ["AE+MoE_F"])

In [405]:
ae_moe[0][t_keys2[0]][0]

array([0.05260848, 0.05117683, 0.05404013])

In [419]:
fig = go.Figure()
swatches = ex.colors.qualitative.Safe
marker_size=10
categories = ["R"+str(i) for i in range(2, 6)]
fig.add_trace(go.Scatterpolar(
    r=[res[t_keys2[0]][0][0] for res in ae_moe],
    theta=categories,
    marker_symbol="cross",
    marker_size=marker_size,
    marker_color=swatches[0],
    name="FE"
    ))
fig.add_trace(go.Scatterpolar(
    r=[res[t_keys2[0]][0][0] for res in rbf_moe],
    theta=categories,
    marker_symbol="square",
    marker_size=marker_size,
    marker_color=swatches[1],
    name="FC"
    ))
fig.add_trace(go.Scatterpolar(
    r=[res[t_keys2[0]][0][0] for res in reduc_moe],
    theta=categories,
    marker_symbol="diamond",
    marker_size=marker_size,
    marker_color=swatches[2],
    name="FS"
    ))
fig.update_layout(
    title_text="(Frozen) Reconstruction error (MSE) over the domains",
    width = 400, height=400,
    legend=dict(orientation="h"), 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)
# if x_axis_name is not None:
#     fig.update_xaxes(
#         title_text=
#     )
fig.update_yaxes(
    title_text="MSE"
)

In [417]:
fig = go.Figure()
swatches = ex.colors.qualitative.Safe
marker_size=10
categories = ["R"+str(i) for i in range(2, 6)]
fig.add_trace(go.Scatterpolar(
    r=[res[t_keys2[2]][0][0] for res in ae_moe],
    theta=categories,
    marker_symbol="cross",
    marker_size=marker_size,
    marker_color=swatches[0],
    name="FE"
    ))
fig.add_trace(go.Scatterpolar(
    r=[res[t_keys2[2]][0][0] for res in rbf_moe],
    theta=categories,
    marker_symbol="square",
    marker_size=marker_size,
    marker_color=swatches[1],
    name="FC"
    ))
fig.add_trace(go.Scatterpolar(
    r=[res[t_keys2[2]][0][0] for res in reduc_moe],
    theta=categories,
    marker_symbol="diamond",
    marker_size=marker_size,
    marker_color=swatches[2],
    name="FS"
    ))
fig.update_layout(
    title_text="Sum of angular update (radian)",
    width = 400, height=400,
    legend=dict(orientation="h"), 
    font_family="Serif", font_size=14, 
    margin_l=5, margin_t=40, margin_b=5, margin_r=5)
# if x_axis_name is not None:
#     fig.update_xaxes(
#         title_text=
#     )
fig.update_yaxes(
    title_text="MSE"
)